# Data Exploration for Loan Default Data Set

In [ ]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import pickle
import matplotlib.pyplot as plt
import numpy as np

from scipy.stats import entropy

%matplotlib inline
output_notebook()

## Data Loading

In [ ]:
train_data = pd.read_csv("../data/loan-default-prediction/train_v2.csv")

In [ ]:
train_data.shape

### Target Column Analysis

In [ ]:
target = train_data["loss"]
target.describe()

In [ ]:
non_zero_default_loss = train_data[train_data["loss"]!=0]
proportion_of_defaults = float(non_zero_default_loss.shape[0])/float(target.shape[0])
print(r"proportion of loans that defaulted: {ratio} %".format(ratio= proportion_of_defaults*100))

In [ ]:
non_zero_default_loss["loss"].plot.hist(bins=20,by="loss", log=True)

### Findings about target variable

About 9.3% of all loans default.
From those the most them default only a small proportion. Except for a full 100% default (which is relatively small proprotion) there seems to be a powerlaw distribution of defaulted value.

## Missing values analysis

In [ ]:
feature_columns = [column for column in train_data.columns.values if column not in ["id", "loss"]]
train_features = train_data[feature_columns]

def get_populated_columns_names(df, threshold):
    dict_for_missing_values = dict(df.isna().any())
    
    columns_with_missing_values = [key for key in dict_for_missing_values if dict_for_missing_values[key]]
    columns_without_missing_values =[key for key in dict_for_missing_values if key not in columns_with_missing_values]
    
    print(r"There are {count} columns with missing values".format(count=len(columns_with_missing_values)))
    print(r"There are {count} columns without missing values".format(count=len(columns_without_missing_values)))

    columns_missing_values_ratio = df.isnull().mean()
    
    return list(columns_missing_values_ratio[columns_missing_values_ratio < threshold].index)
    
train_features = train_features[get_populated_columns_names(train_features, 0.05)]
print(r"Number of features with less then {ratio}% missing values: {count}".format(ratio=0.1*100, count=(train_features.shape[1]-2)))

## Categorical/Numerical column separation

In [ ]:
def get_column_names_by_type(df):
    
    df_data_types = df.dtypes
    numeric_var = [key for key in dict(df_data_types)
                       if dict(df_data_types)[key]
                           in ['float64','float32']]

    int_var = [key for key in dict(df_data_types)
                       if dict(df_data_types)[key]
                           in ['int32','int64']]

    cat_var = [key for key in dict(df_data_types)
                 if dict(df_data_types)[key] in ['object']]
    
    return numeric_var, int_var, cat_var

numeric_var, int_var, cat_var = get_column_names_by_type(train_features)

print(r"There are {count} float type columns".format(count=len(numeric_var)))
print(r"There are {count} int type columns".format(count=len(int_var)))
print(r"There are {count} object type columns".format(count=len(cat_var)))

### Columns With same values

In [ ]:
def get_columns_with_distinct_values(df, column_subset):
    groups = []
    redundant_columns = []
    for i in range(len(column_subset)):
        col1 = column_subset[i]
        if col1 in redundant_columns:
                continue
        same_columns = [col1]
        
        for j in range(i, len(column_subset)):
            col2 = column_subset[j]
            if col1 == col2:
                continue
            if (df[col1]-df[col2]).sum() == 0:
                same_columns += [col2]
                redundant_columns += [col2]
        groups+=[same_columns]
    return [i[0] for i in groups]



In [ ]:
distinct_numeric_var = get_columns_with_distinct_values(train_features, numeric_var)
distinct_int_var = get_columns_with_distinct_values(train_features, int_var)
# distinct_cat_var = get_columns_with_distinct_values(train_features, cat_var)

print(r"There are {count} distinct float type columns".format(count=len(distinct_numeric_var)))
print(r"There are {count} distinct int type columns".format(count=len(distinct_int_var)))
# print(r"There are {count} distinct object type columns".format(count=len(distinct_cat_var)))

### Categorical column analysis

In [ ]:
for cv in cat_var:
    print(r"{cv} has {distinct} distinct values of type {type} and sample value {value}".format(
        cv=cv,
        distinct=len(train_data[cv].unique()),
        type=type(train_data[cv][0]),
        value = train_data[cv][0]
    ))
    

### Numerical column analysis

### Skewness and Standard Devs

In [ ]:
def caluculate_skewness_and_std(df):
    skewnesses = df.skew(axis=0)
    standard_devs = df.std(axis=0)
    return skewnesses, standard_devs

skewnesses, standard_devs = caluculate_skewness_and_std(train_features[distinct_numeric_var + distinct_int_var])

In [ ]:
columns_with_no_standard_dev = list(standard_devs[standard_devs<1e-4].index)

In [ ]:
columns_with_no_standard_dev

In [ ]:
len(train_features.columns.values)

In [ ]:
train_features = train_features.drop(columns=columns_with_no_standard_dev)

In [ ]:
distinct_int_var = [i for i in distinct_int_var if i not in columns_with_no_standard_dev]
distinct_numeric_var = [i for i in distinct_numeric_var if i not in columns_with_no_standard_dev]


In [ ]:
def calculate_entropies_of_numeric_vars(df, columns):
    tmp = [(column, entropy(np.histogram(train_features[column].dropna().values, bins=10000)[0])) for column in columns]
    return sorted(tmp, key = lambda x: x[1])
entropies_of_numeric = calculate_entropies_of_numeric_vars(train_features, distinct_numeric_var)

In [ ]:
entropies_of_numeric

In [ ]:
plt.plot(range(len(entropies_of_numeric)), list(map(lambda x: x[1],entropies_of_numeric)))

In [ ]:
distinct_numeric_var_with_low_entropy = [i[0] for i in entropies_of_numeric if i[1]<4]

### Categorical Columns maskarading as numerical

In [ ]:
def get_sorted_count_of_unique_integer_values(df, column_names):
    unique_integer_values = [(iv, len(df[iv].unique())) for iv in column_names]
    tmp = sorted(unique_integer_values, key = lambda x: x[1])
    
    return tmp

sorted_count_of_unique_integer_values = get_sorted_count_of_unique_integer_values(train_features, distinct_int_var)

In [ ]:
def plot_integer_type_data(df, column_names, columns, rows):
   
    fig, ax = plt.subplots(columns, rows, figsize=(20, 18))
    plt.subplots_adjust(hspace = 0.4, wspace=0.4)
    
    ax = ax.ravel()

    for j,column_name in enumerate(column_names):
        ax[j].hist(df[column_name[0]].values, bins=column_name[1])
        ax[j].set_title(column_name[0])

In [ ]:
plot_integer_type_data(train_features, sorted_count_of_unique_integer_values[:20], 5, 4)

In [ ]:
columns_to_be_converted_to_categorical = ["f776", "f777", "f725", "f2", "f5", "f73", "f403"]

In [ ]:
def calculate_entropies_columns(df, column_names):
    tmp = [(column_name, entropy(df[column_name].value_counts().values)) for column_name in column_names]
    return sorted(tmp, key=lambda x: x[1])
sorted_entropies = calculate_entropies_columns(train_features, distinct_int_var)

In [ ]:
sorted_entropies

In [ ]:
plt.plot(range(len(sorted_entropies)), list(map(lambda x: x[1],sorted_entropies)))

In [ ]:
def calculate_most_frequent_item(df, column_names):
    tmp = [(column_name, df[column_name].value_counts().values[0]) for column_name in column_names]
    return sorted(tmp, key=lambda x: x[1])
most_frequent_items = calculate_most_frequent_item(train_features, distinct_int_var)

In [ ]:
distinct_int_var_with_min_frequency = [i[0] for i in most_frequent_items if i[1] > 5000]

In [ ]:
plt.plot(range(len(most_frequent_items)), list(map(lambda x: x[1],most_frequent_items)))

### Columns to be deleted

* All categorical columns because they have a huge cardinality
* Data columns with more than 10% of missing values
* Data that have 0 standard deviation

In [ ]:
dbfile = open('../data/columns_to_consider', 'ab') 
pickle.dump(dict(
    dictinct_numeric_var=distinct_numeric_var,
    distinct_int_var=distinct_int_var,
    distinct_int_var_with_min_frequency=distinct_int_var_with_min_frequency,
    distinct_numeric_var_with_low_entropy=distinct_numeric_var_with_low_entropy
    ), dbfile)                      
dbfile.close()